# Google Play Store Ratings

descrição do arquivo

## Contextualização
... dataset / fonte / colunas

## Problemática / Objetivo
...

## Setup

...

In [196]:
import tensorflow as tf
import pandas as pd
import numpy as np

from IPython import display

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

googleplaystore_dataframe = pd.read_csv("https://raw.githubusercontent.com/arthurflor/courses/master/google-machine-learning-crash-course/googleplaystore.csv", sep=",")
googleplaystore_dataframe = googleplaystore_dataframe.reindex(np.random.permutation(googleplaystore_dataframe.index))

print("Dataset googleplaystore:")
googleplaystore_dataframe

Dataset googleplaystore:


,app,category,rating,reviews,size,installs,type,price,content_rating,genres,last_uptaded,current_ver,android_ver
10198,Swift for Facebook Lite,SOCIAL,4.3,9562,5.8,500,Free,0.0,Teen,Social,"May 31, 2018",4.1.0,0
4317,K-pop Music,FAMILY,4.0,369,3.5,10,Free,0.0,Everyone,Entertainment,"January 11, 2017",2.0.0,2.3 and up
4131,RPG End of Aspiration F,FAMILY,4.4,20368,31.0,500,Free,0.0,Teen,Role Playing,"December 15, 2016",3.0.0fr,2.3 and up
5575,"AS - Diario online deportivo. Fútbol, motor y más",NEWS_AND_MAGAZINES,0.0,0,6.0,50,Free,0.0,Everyone,News & Magazines,"June 24, 2018",7.8,4.1 and up
5574,MARCA - Diario Líder Deportivo,SPORTS,3.9,76340,14.0,5000000,Free,0.0,Everyone,Sports,"July 25, 2018",6.3.3,4.1 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6897,Watch Face BW Inter,PERSONALIZATION,3.1,112,8.8,1,Paid,1.0,Everyone,Personalization,"April 26, 2017",2.1.7,4.3 and up
5412,Attack the Light,FAMILY,4.7,19209,21.0,100,Paid,3.0,Everyone,Role Playing;Action & Adventure,"February 23, 2016",1.1.4,4.0 and up
6644,Bill Miller Bar-B-Q Store Ops,BUSINESS,0.0,0,2.5,100,Free,0.0,Everyone,Business,"June 19, 2018",1.3,4.4 and up
8405,Castle Defense 2,FAMILY,4.2,35172,34.0,1000000,Free,0.0,Teen,Strategy,"January 20, 2017",3.2.2,2.3.3 and up


...

In [0]:
def preprocess_features(googleplaystore_dataframe):
  """Preparar inputs do dataset googleplaystore.

  Args:
    googleplaystore_dataframe: Pandas DataFrame contendo o dataset.

  Returns:
    DataFrame contendo os atributos para serem usados no modelo.
  """

  selected_features = googleplaystore_dataframe[[
      "category",
      "rating",
      "reviews",
      "size",
      "installs",
      "price",
   ]]

  processed_features = selected_features.copy()
  processed_features["category_code"] = pd.Categorical(processed_features["category"]).codes
  
  # Criando atributo sintético.
  processed_features["rating_per_reviews"] = (googleplaystore_dataframe["rating"] / googleplaystore_dataframe["reviews"])
  
  return processed_features


def preprocess_targets(googleplaystore_dataframe):
  """Preparar labels (targets) do dataset googleplaystore.

  Args:
    googleplaystore_dataframe: Pandas DataFrame contendo o dataset.

  Returns:
    DataFrame contendo as labels (targets).
  """
  
  output_targets = pd.DataFrame()
  
  # Criando valor representando um "app de sucesso" com limiar de rating em 4 e número de reviews em 1000.
  output_targets["success_app"] = ((googleplaystore_dataframe["rating"] > 4,1) and (googleplaystore_dataframe["reviews"] > 1000)).astype(float)
  
  return output_targets

In [208]:
length = len(googleplaystore_dataframe)
training_len = int(length * 0.9)
validation_len = int(length * 0.1)

# Selecionando os primeiros 90% dos dados para treino.
training_examples = preprocess_features(googleplaystore_dataframe.head(training_len))
training_targets = preprocess_targets(googleplaystore_dataframe.head(training_len))

# Selecionando os últimos 10% dos dados para validação.
validation_examples = preprocess_features(googleplaystore_dataframe.tail(validation_len))
validation_targets = preprocess_targets(googleplaystore_dataframe.tail(validation_len))

print("Resumo dos dados de treino:")
display.display(training_examples.describe())

print("Resumo dos dados de validação:")
display.display(validation_examples.describe())

print("Resumo dos dados de treino (labels):")
display.display(training_targets.describe())

print("Resumo dos dados de validação (labels):")
display.display(validation_targets.describe())

Resumo dos dados de treino:


,rating,reviews,size,installs,price,category_code,rating_per_reviews
count,9756.0,9756.0,9756.0,9756.0,9756.0,9756.0,9223.0
mean,3.6,443055.9,31.0,15410168.7,1.1,16.7,0.1
std,1.5,2970231.3,91.0,85061356.0,16.3,8.4,0.5
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,3.7,38.0,3.1,10.0,0.0,11.0,0.0
50%,4.2,2109.5,10.0,100.0,0.0,14.0,0.0
75%,4.5,55387.0,28.0,5000000.0,0.0,25.0,0.0
max,5.0,78158306.0,1020.0,1000000000.0,400.0,32.0,5.0


Resumo dos dados de validação:


,rating,reviews,size,installs,price,category_code,rating_per_reviews
count,1084.0,1084.0,1084.0,1084.0,1084.0,1084.0,1021.0
mean,3.6,454025.7,33.0,15560950.3,0.7,16.9,0.1
std,1.5,2514619.8,98.4,84850337.0,12.3,8.2,0.4
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,3.6,36.8,3.1,10.0,0.0,11.0,0.0
50%,4.2,1896.5,11.0,100.0,0.0,14.0,0.0
75%,4.5,48298.8,29.0,5000000.0,0.0,24.0,0.0
max,5.0,56642847.0,986.0,1000000000.0,400.0,32.0,5.0


Resumo dos dados de treino (labels):


,success_app
count,9756.0
mean,0.5
std,0.5
min,0.0
25%,0.0
50%,1.0
75%,1.0
max,1.0


Resumo dos dados de validação (labels):


,success_app
count,1084.0
mean,0.5
std,0.5
min,0.0
25%,0.0
50%,1.0
75%,1.0
max,1.0


...

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Organizar dados para treinamento.
  
    Args:
      features: pandas DataFrame de atributos
      targets: pandas DataFrame de targets
      batch_size: tamanho do batch para passar ao modelo
      shuffle: Embaralhar os dados ou não.
      num_epochs: Número de épocas. None = indefinido
    
    Returns:
      Tupla de atributos e targets.
    """
    
    features = {key:np.array(value) for key,value in dict(features).items()}                                            
 
    ds = Dataset.from_tensor_slices((features,targets)) # limite de 2GB
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
      ds = ds.shuffle(10000)
    
    features, labels = ds.make_one_shot_iterator().get_next()
    
    return features, labels

...

## Train a Logistic Regression Model and Calculate LogLoss on the Validation Set

...

In [0]:
def train_linear_classifier_model(learning_rate, steps, batch_size, training_examples, training_targets, validation_examples, validation_targets):
  """Trains a linear classification model.
  
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    training_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for validation.
      
  Returns:
    A `LinearClassifier` object trained on the training data.
  """

  periods = 10
  steps_per_period = steps / periods
  
  construct_feature_columns = lambda input_features: set([tf.feature_column.numeric_column(my_feature) for my_feature in input_features])
  
  # Create a linear classifier object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 4.0)  
  linear_classifier = tf.estimator.LinearClassifier(
      feature_columns=construct_feature_columns(training_examples),
      optimizer=my_optimizer
  )
  
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["success_app"], 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["success_app"], 
                                                  num_epochs=1, 
                                                  shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["success_app"], 
                                                    num_epochs=1, 
                                                    shuffle=False)
  
  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("LogLoss (on training data):")
  training_log_losses = []
  validation_log_losses = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    linear_classifier.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.    
    training_probabilities = linear_classifier.predict(input_fn=predict_training_input_fn)
    training_probabilities = np.array([item['probabilities'] for item in training_probabilities])
    
    validation_probabilities = linear_classifier.predict(input_fn=predict_validation_input_fn)
    validation_probabilities = np.array([item['probabilities'] for item in validation_probabilities])
    
    training_log_loss = metrics.log_loss(training_targets, training_probabilities)
    validation_log_loss = metrics.log_loss(validation_targets, validation_probabilities)
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, training_log_loss))
    # Add the loss metrics from this period to our list.
    training_log_losses.append(training_log_loss)
    validation_log_losses.append(validation_log_loss)
  print("Model training finished.")
  
  # Output a graph of loss metrics over periods.
  plt.ylabel("LogLoss")
  plt.xlabel("Periods")
  plt.title("LogLoss vs. Periods")
  plt.tight_layout()
  plt.plot(training_log_losses, label="training")
  plt.plot(validation_log_losses, label="validation")
  plt.legend()

  return linear_classifier

...

In [213]:
linear_classifier = train_linear_classifier_model(
    learning_rate=0.000005,
    steps=100,
    batch_size=10,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Training model...
LogLoss (on training data):


NameError: ignored

## Task 3: Calculate Accuracy and plot a ROC Curve for the Validation Set

A few of the metrics useful for classification are the model [accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision#In_binary_classification), the [ROC curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) and the area under the ROC curve (AUC). We'll examine these metrics.

`LinearClassifier.evaluate` calculates useful metrics like accuracy and AUC.

In [0]:
evaluation_metrics = linear_classifier.evaluate(input_fn=predict_validation_input_fn)

print("AUC on the validation set: %0.2f" % evaluation_metrics['auc'])
print("Accuracy on the validation set: %0.2f" % evaluation_metrics['accuracy'])

AUC on the validation set: 0.69
Accuracy on the validation set: 0.75


You may use class probabilities, such as those calculated by `LinearClassifier.predict`,
and Sklearn's [roc_curve](http://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics) to
obtain the true positive and false positive rates needed to plot a ROC curve.

In [0]:
validation_probabilities = linear_classifier.predict(input_fn=predict_validation_input_fn)
# Get just the probabilities for the positive class.
validation_probabilities = np.array([item['probabilities'][1] for item in validation_probabilities])

false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(
    validation_targets, validation_probabilities)
plt.plot(false_positive_rate, true_positive_rate, label="our model")
plt.plot([0, 1], [0, 1], label="random classifier")
_ = plt.legend(loc=2)

**See if you can tune the learning settings of the model trained at Task 2 to improve AUC.**

Often times, certain metrics improve at the detriment of others, and you'll need to find the settings that achieve a good compromise.

**Verify if all metrics improve at the same time.**

In [0]:
linear_classifier = train_linear_classifier_model(
    learning_rate=0.000003,
    steps=20000,
    batch_size=500,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

evaluation_metrics = linear_classifier.evaluate(input_fn=predict_validation_input_fn)

print("AUC on the validation set: %0.2f" % evaluation_metrics['auc'])
print("Accuracy on the validation set: %0.2f" % evaluation_metrics['accuracy'])